In [18]:
import requests
import urllib
from bs4 import BeautifulSoup
import random
import time
from tqdm import tqdm_notebook as tqdm
import re
import unicodedata
import collections
import re
import string
import json
import pandas as pd

import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

from pymystem3 import Mystem
mystem = Mystem()

In [47]:
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    '''Лемматизация + токенизация'''
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

Проставляем частеречные теги для каждого слова + заменяем теги на понятные для rusvectores

In [21]:
from pymystem3 import Mystem

import requests
import re

url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mapping = {}
r = requests.get(url, stream=True)
for pair in r.text.split('\n'):
    pair = re.sub('\s+', ' ', pair, flags=re.U).split(' ')
    if len(pair) > 1:
        mapping[pair[0]] = pair[1]
        
def tag_mystem(text):  
    m = Mystem()
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            if pos in mapping:
                tagged.append(lemma + '_' + mapping[pos]) # здесь мы конвертируем тэги
            else:
                tagged.append(lemma + '_X') # на случай, если попадется тэг, которого нет в маппинге
        except KeyError:
            continue # я здесь пропускаю знаки препинания, но вы можете поступить по-другому

    return tagged

In [22]:
import numpy as np
from scipy import spatial

model = gensim.models.KeyedVectors.load_word2vec_format('/Users/alinashaymardanova/Downloads/180/model.bin', binary=True)

NB Масштабировать вектор до единичной длины + скалярное произведение векторов

In [23]:
from gensim import matutils

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

Обращение к rusvectores (т.к. очень времязатратно, то не используем)

In [24]:
def api_similarity(m, w1, w2):
    url = '/'.join(['https://rusvectores.org', m, w1 + '__' + w2, 'api', 'similarity/'])
    r = requests.get(url, stream=True)
    return r.text.split('\t')[0]

In [25]:
from scipy.spatial import distance
from scipy.spatial.distance import cosine

Для каждого стихотворения вычисляем попарную близость между каждой парой слов. Используем Евклидову метрику, вычисляем косинус между векторами слов и вычисляем скалярное произведение отмасштабированных векторов.

In [26]:
def get_sim_vect(tag_teat):
    v_sim = list()
    v_cos = list()
    v_eucl = list()
    if len(tag_teat) > 1:
        for word in tag_teat:
            i = -1
            while i <= len(tag_teat):
                i += 1
                try:
                    v_sim.append(similarity(model[word], model[tag_teat[i]]))
                    v_eucl.append(distance.euclidean(model[word], model[tag_teat[i]]))
                    v_cos.append(cosine(model[word], model[tag_teat[i]]))
                except:
                    continue
    
    return  v_sim, v_cos, v_eucl

Выделяем все наречия и прилагательные

In [27]:
def get_ADV(tag_teat):
    adv = [word for word in tag_teat if re.search("ADV|ADJ", word)]
    return adv

Делим на четверостишия

In [28]:
def get_sanz(text):
    n = 5
    text = re.sub('\n \n', ' \n\n', text)
    groups = text.split('\n\n')
    lis = list()
    for el in groups:
        lis.append(normalize(el))
    
    
    return lis

Получаем вектор документа

In [29]:
def get_w2v_vectors(model, lemmas):    
    vec_list = []
    lemmas = tag_mystem(''.join(lemmas))
    for word in lemmas:
        try:
            vec = model[word]
            vec_list.append(vec)
        except:
            continue
    
    if len(vec_list) != 0:
        return (sum(vec_list) / len(vec_list))

Получаем вектор четверостишия

In [30]:
def get_sanz_vectors(vectors):
    v_sim = list()
    v_cos = list()
    v_eucl = list()
    if len(vectors) != 1:
        for word in vectors:
            i = -1
            while i <= len(vectors):
                i += 1
                try:
                    v_sim.append(similarity(word, vectors[i]))
                    v_eucl.append(distance.euclidean(word, vectors[i]))
                    v_cos.append(cosine(word, vectors[i]))
                except:
                    continue

    return v_sim, v_cos,v_eucl

In [31]:
from nltk import ngrams

n-граммы с пробелами

In [32]:
def get_ngram_without_spase(text, N):
    
    ngram = list()
    for word in text.split():
        ngr = ngrams(word, N)
        
        for el in ngr:
            ngram.append(el)
    
    e = collections.Counter(ngram)
    
    return (e[sorted(e, key=e.get, reverse=True)[0]])/len(e)

n-граммы без пробелов

In [33]:
from itertools import islice, tee

def get_ngram(content_norm, N):
    three_grams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(content_norm, N))))
    three_grams = [''.join(x) for x in three_grams]
     
    e = collections.Counter(three_grams)
    return (e[sorted(e, key=e.get, reverse=True)[0]])/len(e)

In [ ]:
new_df = pd.read_csv('/Users/alinashaymardanova/Desktop/person_1.csv', sep='\t', encoding='utf-8')

In [ ]:
texts = new_df['poem']

Полный анализ корпуса стихотворений

In [49]:
def analysis(texts):
    df = pd.DataFrame()
    
    for text in tqdm(texts):
        try:
            clean_text = normalize(text)
            tag_text = tag_mystem(''.join(clean_text))
            v_sim, v_cos, v_eucl = get_sim_vect(tag_text)


            adv = get_ADV(tag_text)
            v_sim_adv, v_cos_adv, v_eucl_adv = get_sim_vect(adv)

            lis = get_sanz(text)
            vectors = list()
            for el in lis:
                vectors.append(get_w2v_vectors(model, el))

            v_sim_f, v_cos_f, v_eucl_f = get_sanz_vectors(vectors)

            bag = collections.Counter(re.findall(r'\w+', clean_text))
            best = sorted(bag, key=bag.get, reverse=True)[:1] 
            res = bag[''.join(best)]

            three = get_ngram(clean_text, 2)
            two = get_ngram(clean_text, 3)
            
            two_gram = get_ngram_without_spase(clean_text, 2)
            three_gram = get_ngram_without_spase(clean_text, 3)
            two_gram_s = get_ngram(clean_text, 2)
            three_gram_s = get_ngram(clean_text, 3)
            
            
            df =  df.append({'Author': 'Person', 
                             'Poem': text,
                             'Semantic_density_dot': np.mean(v_sim),
                             'Semantic_density_cos': np.mean(v_cos),
                             'Semantic_density_eucl': np.mean(v_eucl),
                             'Adj_semantic_density_dot': np.mean(v_sim_adv),
                             'Adj_semantic_density_cos': np.mean(v_cos_adv),
                             'Adj_semantic_density_eucl': np.mean(v_eucl_adv),
                             'Stanzas_dot': np.mean(v_sim_f),
                             'Stanzas_cos': np.mean(v_cos_f),
                             'Stanzas_eucl': np.mean(v_eucl_f),
                             'Frequency word': res,
                             'Bigramms_without_spaces': two_gram,
                             'ThreeGrams_without_spaces': three_gram,
                            'Bigramms_with_spaces': two_gram_s,
                            'ThreeGrams_with_spaces': three_gram_s,
                            'Len': len(text.split())}, 
                            ignore_index=True)
        except:
            continue
        
    return df

In [ ]:
anal_df = df['poem']

In [ ]:
# two_gram = list()
# for poe in tqdm(anal_df):
#     try:
#         clean_text = normalize(poe)
#         two_gram.append(get_ngram_without_spase(clean_text, 2))
#     except:
#         print(poe)
        
# three_gram = list()
# for poe in tqdm(anal_df):
#     try:
#         clean_text = normalize(poe)
#         three_gram.append(get_ngram_without_spase(clean_text, 3))
#     except:
#         print(poe)

# two_gramm = list()

# for poe in tqdm(anal_df):
#     try:
#         clean_text = normalize(poe)
#         two_gramm.append(get_ngram(clean_text, 2))
#     except:
#         print(poe)
        
        
# three_gramm = list()

# for poe in tqdm(anal_df):
#     try:
#         clean_text = normalize(poe)
#         three_gramm.append(get_ngram(clean_text, 3))
#     except:
#         print(poe)

In [52]:
df['Stanzas_dot'].fillna((df['Stanzas_dot'].mean()), inplace=True)
df['Stanzas_eucl'].fillna((df['Stanzas_eucl'].mean()), inplace=True)
df['Stanzas_sim'].fillna((df['Stanzas_sim'].mean()), inplace=True)
df['Adj_semantic_density_dot'].fillna((df['Adj_semantic_density_dot'].mean()), inplace=True)
df['Adj_semantic_density_eucl'].fillna((df['Adj_semantic_density_eucl'].mean()), inplace=True)
df['Adj_semantic_density_sim'].fillna((df['Adj_semantic_density_sim'].mean()), inplace=True)
df['Semantic_density_dot'].fillna((df['Semantic_density_dot'].mean()), inplace=True)
df['Semantic_density_eucl'].fillna((df['Semantic_density_eucl'].mean()), inplace=True)
df['Semantic_density_sim'].fillna((df['Semantic_density_sim'].mean()), inplace=True)

In [72]:
df.to_csv('/Users/alinashaymardanova/Desktop/result.csv', sep='\t', encoding='utf-8')

In [44]:
df = pd.read_csv('/Users/alinashaymardanova/Desktop/result.csv', sep='\t', encoding='utf-8')

In [73]:
df.head()

,Author,Poem,Adj_semantic_density_cos,Adj_semantic_density_eucl,Adj_semantic_density_dot,Semantic_density_cos,Semantic_density_eucl,Semantic_density_dot,Frequency word,Stanzas_dot,Stanzas_eucl,Stanzas_cos,Bigramms_without_spaces,ThreeGrams_without_spaces,Bigramms_with_spaces,ThreeGrams_with_spaces,Len
0,Computer,"Он был похож на скрежет тормозов,\nОна синюшны...",0.594242,20.420713,0.405758,0.883234,35.879446,0.116766,1.0,0.505724,10.195531,0.494276,0.046154,0.032787,0.050633,0.032258,20
1,Computer,"Задрав вершины милые штрихи,\nОтрыв седой равн...",0.567463,22.646386,0.432537,0.877236,33.235625,0.122764,1.0,0.505724,10.195531,0.494276,0.043478,0.044776,0.056818,0.028571,17
2,Computer,"Дренажных сверстников сбивая,\nНога юлила меж ...",0.748438,23.293593,0.251562,0.903880,31.773463,0.096120,2.0,0.505724,10.195531,0.494276,0.107527,0.069307,0.087719,0.064516,28
3,Computer,Краснощёкий кучер подбоченясь\nСрезал курящих ...,0.632428,15.204069,0.367572,0.880547,28.207399,0.119453,1.0,0.505724,10.195531,0.494276,0.058824,0.031746,0.048780,0.021053,16
4,Computer,Импозантные сосульки\nразукрасили брикеты\nСоч...,0.748760,24.449864,0.251240,0.872494,30.420635,0.127506,2.0,0.505724,10.195531,0.494276,0.063158,0.028571,0.072072,0.039735,25


In [54]:
df.loc[:, df.isnull().any()].head()

""
0
1
2
3
4
